In [1]:
# Dependencies

import json
import requests
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go

In [2]:
# Get data from website and save_to_json_file()
def fetch_world_temperatures():
    url = "https://climatereanalyzer.org/clim/t2_daily/json/cfsr_world_t2_day.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

def save_to_json_file(data, filename):
    with open(filename, "w") as json_file:
        json.dump(data, json_file)

if __name__ == "__main__":
    world_temperatures = fetch_world_temperatures()
    if world_temperatures is not None:
        save_to_json_file(world_temperatures, "Resources\world_2m_temps.json")
        print("Data successfully fetched and saved to 'world_2m_temps.json' file.")
        

Data successfully fetched and saved to 'world_2m_temps.json' file.


In [3]:
import pandas as pd

def load_json_file(filename):
    with open(filename, "r") as json_file:
        data = json.load(json_file)
        return data

data = load_json_file("Resources\world_2m_temps.json")

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Convert the "data" column to a list of lists
df['data'] = df['data'].apply(lambda x: x if isinstance(x, list) else [])

# Create a new DataFrame to store the monthly averages
monthly_averages_df = pd.DataFrame()

# Loop through each row of the original DataFrame and calculate monthly averages
for index, row in df.iterrows():
    name = row['name']
    data_list = row['data']
    
    # Split the data into chunks of 30 days (assuming there are 30 days per month)
    monthly_data_chunks = [data_list[i:i + 30] for i in range(0, len(data_list), 30)]
    
    # Calculate the monthly averages (handling None values)
    monthly_averages = [sum(chunk) / len(chunk) if chunk and None not in chunk else None for chunk in monthly_data_chunks]
    
    # Create a dictionary to store the monthly average data for this "name"
    monthly_average_data = {
        'name': [name] * len(monthly_averages),
        'month': [f"Month {i+1}" for i in range(len(monthly_averages))],
        'monthly_average': monthly_averages
    }
    
    # Convert the dictionary to a DataFrame and append it to the main monthly_averages_df
    monthly_averages_df = monthly_averages_df.append(pd.DataFrame(monthly_average_data))

# Reset the index of the monthly_averages_df
monthly_averages_df.reset_index(drop=True, inplace=True)

# Display the final DataFrame with monthly averages
monthly_averages_df


,name,month,monthly_average
0,1979,Month 1,12.240933
1,1979,Month 2,12.183800
2,1979,Month 3,13.108233
3,1979,Month 4,14.007233
4,1979,Month 5,15.076267
...,...,...,...
619,minus 2σ,Month 9,15.242333
620,minus 2σ,Month 10,14.200000
621,minus 2σ,Month 11,13.010333
622,minus 2σ,Month 12,12.285333


In [4]:
# Import the calendar module
import calendar

# Define a function that takes a month number and returns the month name
def get_month_name(month_number): # Use the calendar module to get the month name return calendar.month_name[int(month_number)]

# Apply the function to the month column of the monthly_averages_df
    monthly_averages_df['month'] = monthly_averages_df['month'].apply(lambda x: get_month_name(x.split()[1]))

# Display the updated DataFrame 
monthly_averages_df

,name,month,monthly_average
0,1979,Month 1,12.240933
1,1979,Month 2,12.183800
2,1979,Month 3,13.108233
3,1979,Month 4,14.007233
4,1979,Month 5,15.076267
...,...,...,...
619,minus 2σ,Month 9,15.242333
620,minus 2σ,Month 10,14.200000
621,minus 2σ,Month 11,13.010333
622,minus 2σ,Month 12,12.285333


In [5]:
# Split off the number of the month into a new column named month_number
monthly_averages_df['month_number'] = monthly_averages_df['month'].apply(lambda x: x.rsplit(" ", 1)[-1])

# Drop the original month column
monthly_averages_df.drop('month', axis=1, inplace=True)


# Display the updated DataFrame
monthly_averages_df 

import calendar

monthly_averages_df['calendar_month'] = monthly_averages_df['month_number'].apply(calendar.month_name)

TypeError: '_localized_month' object is not callable

In [ ]:
import calendar

def load_json_file(filename):
    with open(filename, "r") as json_file:
        data = json.load(json_file)
        return data

data = load_json_file("Resources\world_2m_temps.json")

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Convert the "data" column to a list of lists
df['data'] = df['data'].apply(lambda x: x if isinstance(x, list) else [])

# Create a new DataFrame to store the monthly averages
monthly_averages_df = pd.DataFrame()

# Loop through each row of the original DataFrame and calculate monthly averages
for index, row in df.iterrows():
    name = row['name']
    data_list = row['data']
    
    # Split the data into chunks of 30 days (assuming there are 30 days per month)
    monthly_data_chunks = [data_list[i:i + 30] for i in range(0, len(data_list), 30)]
    
    # Calculate the monthly averages (handling None values)
    monthly_averages = [sum(chunk) / len(chunk) if chunk and None not in chunk else None for chunk in monthly_data_chunks]
    
    # Create a dictionary to store the monthly average data for this "name"
    valid_months = [calendar.month_name[i] for i in range(1, len(monthly_averages) + 1) if monthly_averages[i-1] is not None]
    valid_averages = [avg for avg in monthly_averages if avg is not None]
    
    monthly_average_data = {
        'name': [name] * len(valid_averages),
        'month': valid_months,
        'monthly_average': valid_averages
    }
    
    # Convert the dictionary to a DataFrame and append it to the main monthly_averages_df
    monthly_averages_df = monthly_averages_df.append(pd.DataFrame(monthly_average_data))

# Reset the index of the monthly_averages_df
monthly_averages_df.reset_index(drop=True, inplace=True)

# Display the final DataFrame with monthly averages
print(monthly_averages_df)